## Jigsaw TPU:XLM-Roberta
- 주요 초점 영역은  `독성을 식별할 수 있는 머신러닝 모델`
- 독성은 무례하거나 무례한 것으로 정의되거나 
- 누군가 토론을 떠날 가능성이 있는 것으로 정의된다.
- 만약 이러한 독성 기여가 확인된다면, 우리는 더 안전하고, 더 협력적인 인터넷을 가질 수 있을 것이다.
- 올해는 새로운 TPU 지원을 활용, 
- `영어 전용 훈련 데이터`로 `다국어 모델 구축`에 도전한다.

In [5]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

## Functions

In [3]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    #chunk_size = 256 .. 큰 덩어리 사이즈
    # tokenizer의 라이브러리 사용
    # 각각 문장의 시작 부분이나 끝 부분 또는 단어 시퀀스 중 
    # maxlen(=512)보다 큰 시퀀스에서 값을 제거한다.
    tokenizer.enable_truncation(max_length=maxlen)
    # padding: 'pre' or 'post', pad either before or after each sequence.
    # max_sentences: The max sentence length to use.
    # 문장 길이는 최대 512로
    tokenizer.enable_padding(max_length=maxlen)

    all_ids = []
    # texts의 처음부터 끝까지 chunk_size(=256)씩 건너띄면서
    for i in tqdm(range(0, len(texts), chunk_size)):
        # text_chunk는 해당 인덱스에서 256개문자의 data
        # tolist() = data를 list형태로
        text_chunk = texts[i:i+chunk_size].tolist()
        # tokenize multiple sentences at once;
        # 한번에 여러문장들을 토큰화
        #....백만건에 문장을 10초로
        encs = tokenizer.encode_batch(text_chunk)
        # encs의 문자들이 all_ids에 덮어진다
        all_ids.extend([enc.ids for enc in encs])
        
    # 배열 생성함수
    # 순서가 있는 객체(주로 리스트)를 넘겨받아 데이터가 들어있는 새로운 NumPy 배열 생성
    return np.array(all_ids)

In [6]:
# 인코드 작업은 단어를 벡터(Vector)로 변환해 단어의 의미를 캡슐화하는 것으로, 
# 비슷한 단어의 숫자가 더 가깝다.
def regular_encode(texts, tokenizer, maxlen=512):
    # encode the word to vector of integer
    # 단어를 정수의 벡터로 인코딩
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        # masking 처리가 된 토큰들에게 attention점수를 준 값 반환 X
        return_attention_masks=False, 
        # type_ids = 형식 식별자 목록
        # string_id 색인 기준으로 정렬해야하며 중복항복이 포함되지 않는 목록
        return_token_type_ids=False,
        # First sentence will have some PADDED tokens to match second sequence length
        # 첫 번째 문장은 두 번째 시퀀스 길이와 일치하는 PADED 토큰이 (TRUE) 있을 것이다.
        pad_to_max_length=True,
        max_length=maxlen   // 512
    )
    # 배열 생성함수
    # 순서가 있는 객체(주로 리스트)를 넘겨받아 데이터가 들어있는 새로운 NumPy 배열 생성
    return np.array(enc_di['input_ids'])

## Bert 언어모델
- Bi-directional Encoder Representations from Transformers

`간단한 설명`
![image.png](https://lh3.googleusercontent.com/proxy/Qa_FsiWCU2pwdpnKKAhjJAaUZdB9P0HOm3BUXVx0_Ycomo-YYBEyWRe5LP6bSfpj2X9QSinZM9GeEJntjNAy4E4Pg66gCu1MottnGbqbsNZOuEwjbJAUhR81p3JrhuIhHwf88OMq00cmWYWPXHKOOg)

- input은 sectence 문장 두개로 입력
- sentence는 token 단위로 imbedded
- imbedded된 것은 transformer layer 12개를 거친 후에 본인을 표현

![image.png](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)
- cls

In [7]:
def build_model(transformer, max_len=512):
    #Input: for define input layer 
    #shape is vector with 512-dimensional vectors
    #512차원 벡터를 가진 텐서 반환
    # dtype(data type) : tf.int32 (int32 정수)
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids") # 입력 layer 정의
    sequence_output = transformer(input_word_ids)[0]
     
    # to get the vector
    # cls
    # 모든 sentence의 첫번째 token은 언제나 cls를 갖는다
    # 문장의 시작을 알리는 토큰
    # transformer의 encoding을 마치면 cls토큰은 
    # 나머지 이후 토큰들의 의미들을 응축하게 된다
    cls_token = sequence_output[:, 0, :]
    
    # define output layer
    # Dense layer : 모든 입력 뉴런과 출력 뉴런을 연결하는 전 결합층
    # activation = 'sigmond'함수는 활성화 함수로
    # 입력되는 값을 0 과 1 사이의 값으로 출력 됩니다.
   
    out = Dense(1, activation='sigmoid')(cls_token)
    # cls token과 레이어가 매끄럽게 학습될 수 있도록 한다
    # initiate the model with inputs and outputs
    
    model = Model(inputs=input_word_ids, outputs=out)
    # model.compile()에서 Adam optimizer 사용
    # lr: 0보다 크거나 같은 float 값. 학습률.
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# TPU Configs

## Run Bert Model on TPU

**데이터의 tokenizing**

ex) He likes playing -> He likes play ##ing

- 입력문장을 tokenizing하고, 빈도수에 따라 분리
- input 문장 두개의 token sequence 가 학습에 사용
- 위의 그림 참고
![image.png](https://mino-park7.github.io/images/2019/02/bert-input-representation.png)

### 전처리 이후 tokenize 필요
- 빈도수에 기반해 단어를 의미있는 패턴으로 잘라서 tokenizing = word piece tokenizing

*1) 빈도수를 검색해서 vocab후보를 계속해서 update -> 최종*

*2) BERT 뒷단어에 ##을 붙여서 구별 (같은 글자라고 맨앞과 앞이 아닌거는 다르므로)*

*3) vocab 후보를 기준으로 Bigrampairs를 만들고 빈도수를 계산해 가장 많이 등장한 Best pair를 하나로 합친다*

*4) 이 과정을 반복해 최종 vocab에 리스트 저장-> 정해진 iteration 모두 수행*

### INPUT
앞에는 [cls] dog is pretty 뒤에는 [sep] 토큰

*1) 입력token에 대한 word imbedding*

*2) segment imbedding은 첫번째 문장인지, 두번째 문장인지 구별*

*3) position imbedding 빈도의 개수에 따라 position imbedding을 다르게 해서 학습*

*BERTbase = 4개의 TPU사용*


In [8]:
# Detect hardware, return appropriate distribution strategy
# TPU :Tensor Processing Unit
# 구글에서 2016년 5월에 발표한 데이터 분석 및 딥러닝용 하드웨어이다.
# CPU나 GPU에서 학습시킬 때와는 비교도 안되는 수준의
# 학습 속도를 보입니다
# TPU의 사용방법은 크게 두가지
# 1) TPUstimator 와 2) TPUstrategy
# 학습 중 상태 값들과 계산 워크로드의 분산을 위해 설정하는 정책
# 객체를 활용해서 데이터나 연산 부담을 여러기기에 분산

try:  # 실행할 코드
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # TPUClusterResolver : 하나 혹은 여러 대의 TPU 집합을 잡는다
    # tpu : 문자열 또는 사용할 TPU에 해당하는 문자열 목록. 
    # 단일 문자열이 빈 문자열, 'local' 문자열 또는 'grpc://' 또는 '/bns'로 시작하는 문자열인 경우, 
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master()) 
    # grpc경로 반환 : grpc://10.0.0.2:8470
except ValueError: # 예외가 발생했을 때 처리하는 코드
    tpu = None

if tpu:
    # experimental_connect_to_cluster 함수와 
    # initialize_tpu_system 함수를 통해 TPU와 런타임을 연결한다
    
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # 그 후에는 strategy를 생성
    # TPU의 집합을 데이터 분산 대상에 넣어준다
    # tf.distribute.experimental.TPUStrategy
    # 텐서 플로 훈련을 텐서처리장치에서 수행하는 전략
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()
# 복제본의 수
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [9]:
# AUTO = tf.data.experimental.AUTOTUNE
# 작동하는 Network가 스스로 설정하고 
# Dataset을 잘 불러올 수 있게 결정
# 많은 양의 Data를 처리하는 경우 Load를 자동으로 해주기 때문에
# 속도가 빨라질 것으로 예상
AUTO = tf.data.experimental.AUTOTUNE

# Data access
#GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
# EPOCHS 는 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것
# 전체 데이터 셋에 대해 한번 학습을 완료한 상태
EPOCHS = 2 
# 전체 데이터를 2번 사용해서 학습을 거치는 것
# BATCH_SIZE : 한 번의 batch마다 주는 데이터 샘플의 size
BATCH_SIZE = 16 * strategy.num_replicas_in_sync # 복제본 8
MAX_LEN = 192 
# XLM-Roberta 모델 이용 ( bert보다 성능 우수)
# 변압기 아키텍처 훈련과 아키텍처의 확장 우수
MODEL = 'jplu/tf-xlm-roberta-large'

![image](https://mblogthumb-phinf.pstatic.net/MjAxOTAxMjNfMjU4/MDAxNTQ4MjM1Nzg3NTA2.UtvnGsckZhLHOPPOBWH841IWsZFzNcgwZvYKi2nxImEg.CdtqIxOjWeBo4eNBD2pXu5uwYGa3ZVUr8WZvtldArtYg.PNG.qbxlvnf11/20190123_182720.png?type=w800)

In [10]:
# First load the real tokenizer 실제의 tokenizer 로드
# 토큰화(문자열 분할), 토큰 문자열 인코딩과 디코딩(정수로 변환)
# 새로운 토큰들 추가, 마스크 관리 등 이루어진다
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [17]:
# 각데이터에서 중요한 것은 comment_text 칼럼
# 독성이 있는 것 or 독성이 없는 것으로 분류된 텍스트
# 독성이 있는 논평은 1점을 받고, 독성이 없는 논평은 0점

train1 = pd.read_csv("./jigsaw-toxic-comment-train.csv")
train2 = pd.read_csv("./jigsaw-unintended-bias-train.csv")
# train2의 toxic값을 반올림->int 형변환
train2.toxic = train2.toxic.round().astype(int)

valid = pd.read_csv('./validation.csv')
test = pd.read_csv('./test.csv')
sub = pd.read_csv('./sample_submission.csv')

In [22]:
# Combine train1 with a subset of train2
train = pd.concat([
    train1[['comment_text', 'toxic']],
    # query : 조건식 (문자열)
    train2[['comment_text', 'toxic']].query('toxic==1'),
    # n = 추출할 샘플의 수
    # random_state : 랜덤 샘플 추출 시 시드 입력
    train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000, random_state=0)
])

train.tail()

,comment_text,toxic
536378,"""Natural justice"" and ""natural rights"" are ver...",0
486306,"""Like a bad food critic, the MSM concentrated ...",0
180553,"he didn't mean it, he's sorry so back off please",0
32462,The legislators have a lot on their plate to d...,0
921647,Let's assume that there is only so much money ...,0


In [26]:
%%time 
# regul_encode 함수 : 단어를 정수의 벡터(Vector)로 변환해 단어의 의미를 캡슐화
x_train = regular_encode(train.comment_text.values, tokenizer, maxlen = MAX_LEN)
x_valid = regular_encode(valid.comment_text.values, tokenizer, maxlen = MAX_LEN)
x_test = regular_encode(test.content.values, tokenizer, maxlen = MAX_LEN)

y_train = train.toxic.values
y_valid = valid.toxic.values

AssertionError: 

In [28]:
# TPU는 하나의 학습 단계를 실행하는데 필요한 시간을 급격하게 감소
# 최대 성능 위해 현재 단계가 종료되기 전에 다음 스텝의 데이터를 운반하는 효율적인 입력 파이프라인이 필요
# tf.data API는 유연하고 효율적인 입력 파이프라인을 만드는데 도움이 됩니다

# Dataset 사용하려면
# 1) 데이터 불러오기
# 2) Iterator(반복자) 생성하기
# 3) 데이터 사용하기
train_dataset = (
    # x_train과 y_train으로 dataset 생성
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    # repeat(step_n) : 원하는 epoch 수를 넣을 수 있음
    .repeat() # 파라미터 X : iteration이 무제한으로 돌아감
    .shuffle(2048) # 한번 epoch가 돌고나서 랜덤하게 섞을 것 정함
    .batch(BATCH_SIZE)
    .prefetch(AUTO) # 연산에 필요한 data들을 미리 가져옴
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    # .cashe() : preprocessing 시간이 너무 길어서 줄이고 싶을때 사용
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

NameError: name 'x_train' is not defined

In [29]:
%%time
# 이전에 distribute Strategy
# 모델을 불어올 때 범위(scope)를 지정해도 되고 안해도 됨 
with strategy.scope():
    # 사전 교육된 MODEL에서 TF AutoModel 구현
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    # build model 함수 호출
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
Wall time: 6min 48s


In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
# 모델 학습
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps, # 한 epoch에 사용한 스텝 수
    validation_data=valid_dataset,
    epochs=EPOCHS  # 2
)

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    # .repeat() 원하는 epoch 수를 넣을 수 있음
    valid_dataset.repeat(),
    steps_per_epoch=n_steps, # 한 epoch에 사용한 스텝 수
    epochs=EPOCHS # 2
)

In [ ]:
# model.predict : 트레이닝이 끝난 모델을 분류수행
sub['toxic'] = model.predict(test_dataset, verbose=1)
sub.to_csv('submission.csv', index=False)